In [2]:
import os
from pathlib import Path
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
from abc import ABC, abstractmethod
from transformers import ViTForImageClassification, ViTImageProcessor
import torch
import numpy as np
import matplotlib.pyplot as plt

from dotenv import load_dotenv

# 1) Load .env
load_dotenv()

class PipelineStep(ABC):
    """Abstract base class for pipeline steps."""
    
    @abstractmethod
    def process(self, data):
        """Process the input data and return the transformed output."""
        pass

class AllenAPI:
    """Singleton class for accessing Allen Brain Observatory API"""
    
    _instance = None
    _boc = None  # Lazy-loaded BrainObservatoryCache instance

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    @property
    def boc(self):
        """Lazy-load BrainObservatoryCache only when accessed."""
        if self._boc is None:
            allen_cache_path = os.environ.get('CAIM_ALLEN_CACHE_PATH')
            if not allen_cache_path:
                raise ValueError("AllenAPI requires a valid cache path. Set `CAIM_ALLEN_CACHE_PATH` in .env.")

            manifest_path = Path(allen_cache_path) / 'brain_observatory_manifest.json'
            self._boc = BrainObservatoryCache(manifest_file=str(manifest_path))

        return self._boc

    def get_boc(self):
        """Retrieve the BrainObservatoryCache object, ensuring it is initialized."""
        return self.boc
import os
from pathlib import Path
from allensdk.core.brain_observatory_cache import BrainObservatoryCache

class AllenAPI:
    """Singleton class for accessing Allen Brain Observatory API"""
    
    _instance = None
    _boc = None  # Lazy-loaded BrainObservatoryCache instance

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    @property
    def boc(self):
        """Lazy-load BrainObservatoryCache only when accessed."""
        if self._boc is None:
            allen_cache_path = os.environ.get('CAIM_ALLEN_CACHE_PATH')
            if not allen_cache_path:
                raise ValueError("AllenAPI requires a valid cache path. Set `CAIM_ALLEN_CACHE_PATH` in .env.")

            manifest_path = Path(allen_cache_path) / 'brain_observatory_manifest.json'
            self._boc = BrainObservatoryCache(manifest_file=str(manifest_path))

        return self._boc

    def get_boc(self):
        """Retrieve the BrainObservatoryCache object, ensuring it is initialized."""
        return self.boc

# Create a global instance so that all files can use it
allen_api = AllenAPI()

class AllenStimuliFetchStep(PipelineStep):
    """
    Fetches data from the Allen Brain Observatory.
    The session IDs are hard-coded since the stimuli are always the same.
    """
    # Hard-coded sessions
    SESSION_A = 501704220
    SESSION_B = 501559087
    SESSION_C = 501474098

    def __init__(self, boc):
        """
        :param boc: BrainObservatoryCache instance (via the AllenAPI singleton).
        """
        self.boc = boc

    def process(self, data):
        """
        Expects data to be either None or have (container_id, session, stimulus).
        We fetch a dictionary of raw stimuli arrays, store them in data['raw_data_dct'].
        """
        if isinstance(data, tuple):
            container_id, session, stimulus = data
            data = {'container_id': container_id, 'session': session, 'stimulus': stimulus}
        elif data is None:
            data = {}

        raw_data_dct = {}

        movie_one_dataset = self.boc.get_ophys_experiment_data(self.SESSION_A)
        raw_data_dct['natural_movie_one'] = movie_one_dataset.get_stimulus_template('natural_movie_one')

        movie_two_dataset = self.boc.get_ophys_experiment_data(self.SESSION_C)
        raw_data_dct['natural_movie_two'] = movie_two_dataset.get_stimulus_template('natural_movie_two')

        movie_three_dataset = self.boc.get_ophys_experiment_data(self.SESSION_A)
        raw_data_dct['natural_movie_three'] = movie_three_dataset.get_stimulus_template('natural_movie_three')

        natural_images_dataset = self.boc.get_ophys_experiment_data(self.SESSION_B)
        raw_data_dct['natural_scenes'] = natural_images_dataset.get_stimulus_template('natural_scenes')

        data['raw_data_dct'] = raw_data_dct
        return data

class AnalysisPipeline:
    """Executes a series of PipelineStep objects in order."""
    
    def __init__(self, steps):
        self.steps = steps
    
    def run(self, data):
        for step in self.steps:
            data = step.process(data)
        return data

# Create a global instance so that all files can use it
allen_api = AllenAPI()
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")

    # F) Build pipeline with all steps
boc = allen_api.get_boc()
pipeline = AnalysisPipeline([
        AllenStimuliFetchStep(boc)])

raw_data_dct=pipeline.run(None)
'''
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])'''

/home/maria/NeuroGarage/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


'\ninputs = processor(images=image, return_tensors="pt")\noutputs = model(**inputs)\nlogits = outputs.logits\n# model predicts one of the 1000 ImageNet classes\npredicted_class_idx = logits.argmax(-1).item()\nprint("Predicted class:", model.config.id2label[predicted_class_idx])'

In [3]:
images=raw_data_dct['raw_data_dct']['natural_scenes']

print(images.shape)

(118, 918, 1174)


In [5]:
print(images)

[[[165. 165. 165. ...  69.  54.  54.]
  [165. 165. 165. ...  69.  54.  54.]
  [165. 165. 165. ...  69.  54.  54.]
  ...
  [144. 144. 144. ... 161. 197. 197.]
  [131. 131. 131. ... 231. 244. 244.]
  [131. 131. 131. ... 231. 244. 244.]]

 [[ 77.  77.  77. ...  79.  95.  95.]
  [ 77.  77.  77. ...  79.  95.  95.]
  [ 77.  77.  77. ...  79.  95.  95.]
  ...
  [ 63.  63.  63. ...  86.  87.  87.]
  [ 70.  70.  70. ...  47.  51.  51.]
  [ 70.  70.  70. ...  47.  51.  51.]]

 [[193. 193. 193. ... 178. 186. 186.]
  [193. 193. 193. ... 178. 186. 186.]
  [193. 193. 193. ... 178. 186. 186.]
  ...
  [176. 176. 176. ... 218. 215. 215.]
  [173. 173. 173. ... 211. 211. 211.]
  [173. 173. 173. ... 211. 211. 211.]]

 ...

 [[ 64.  64.  64. ...  33.  33.  34.]
  [ 64.  64.  64. ...  33.  33.  34.]
  [ 61.  61.  61. ...  35.  35.  33.]
  ...
  [128. 128. 129. ...  85.  85.  82.]
  [128. 128. 129. ...  85.  85.  82.]
  [123. 123. 129. ...  84.  84.  84.]]

 [[ 21.  21.  29. ... 107. 107. 112.]
  [ 21.  21.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image

def classify_image(image):
    # Convert NumPy image to PIL format
    image_pil = Image.fromarray((image).astype(np.uint8))  # Ensure proper format

    # Convert to PyTorch tensor and process
    inputs = processor(images=image_pil, return_tensors="pt")
    
    # Ensure model is on the same device as inputs
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_idx = logits.argmax(-1).item()
    
    return model.config.id2label[predicted_class_idx]

# Convert grayscale to RGB
images_rgb = np.repeat(images[:, :, :, np.newaxis], 3, axis=-1)  # Shape: (118, 918, 1174, 3)

# Loop over images, classify, and display one at a time
for i in range(118):
    image = images_rgb[i]
    
    label = classify_image(image)
    print(label)
    
    #plt.imshow(image / 255)  # Ensure correct range for display
    #plt.axis("off")
    #plt.show()


brown bear, bruin, Ursus arctos
brown bear, bruin, Ursus arctos
brown bear, bruin, Ursus arctos
mud turtle
albatross, mollymawk
lion, king of beasts, Panthera leo
lion, king of beasts, Panthera leo
lion, king of beasts, Panthera leo
tusker
tusker
zebra
tiger, Panthera tigris
tiger, Panthera tigris
cougar, puma, catamount, mountain lion, painter, panther, Felis concolor
coyote, prairie wolf, brush wolf, Canis latrans
cheetah, chetah, Acinonyx jubatus
leopard, Panthera pardus
leopard, Panthera pardus
bald eagle, American eagle, Haliaeetus leucocephalus
kite
canoe
pelican
leopard, Panthera pardus
baboon
hummingbird
otter
whiptail, whiptail lizard
zebra
quail
red wolf, maned wolf, Canis rufus, Canis niger
worm fence, snake fence, snake-rail fence, Virginia fence
megalith, megalithic structure
gazelle
Siberian husky
tusker
kite
ram, tup
valley, vale
hay
Egyptian cat
ladybug, ladybeetle, lady beetle, ladybird, ladybird beetle
monarch, monarch butterfly, milkweed butterfly, Danaus plexippus
l